## Punto 4c:

## Python 

Comenzamos utilizando python para implementar la solución del punto 4c, en este punto usamos factorización PLU dado que Matlab de fondo al hacer LU realmente hace pivoteo parcial

In [ ]:
import numpy as np
from scipy.linalg import cholesky, solve_triangular, hilbert, lu

n = 10
H = hilbert(n) 
b = np.array([1 / (i + n-1) for i in range(1, n + 1)])
P, L, U = lu(H)  

y = solve_triangular(L, P.T@b, lower=True)
a= solve_triangular(U, y)

print(a)


[ 0.  0.  0.  0. -0.  0. -0.  0. -0.  1.]


Implementamos también Cholesky

In [ ]:
import numpy as np
from scipy.linalg import cholesky, solve_triangular, hilbert, lu


n = 10
H = hilbert(n) 
b = np.array([1 / (i + n-1) for i in range(1, n + 1)])
L = cholesky(H, lower=True)

y = solve_triangular(L, b, lower=True)
a= solve_triangular(L.T, y)

print(a)

[ 1.70086722e-11 -1.50685462e-09  3.26950703e-08 -3.01492436e-07
  1.45436401e-06 -4.03474038e-06  6.66994533e-06 -6.48671365e-06
  3.42388651e-06  9.99999244e-01]


Se evidencia que estamos aproximando el polinomio por el propio polinomio, de hecho esto se ve mejor cuando hacemos Gauss, allí el error de hecho es menor. Esto se justificará mejor en el PDF

## Matlab

Comparamos con Matlab

In [ ]:
n = 10; 
H = hilb(n); %Genera la matriz de Hilbert de orden n
b = zeros(n, 1); %Crea un vector con n ceros 
for i = 1:n
    b(i) = 1 / (i + n - 1); %Cambia  las entradas por las del ejercicio
end
[L, U] = lu(H);

%Solucionamos los sistemas

y = L \ b;
a_LU = U \ y;

disp(a_LU')

   0   0   0   0   0   0   0   0   0   1


In [21]:
n = 10; 
H = hilb(n); 

b = zeros(n, 1); 
for i = 1:n
    b(i) = 1 / (i + n - 1); 
end

L = chol(H); 

y = L' \ b;
x = L \ y;

disp(x')

 Columns 1 through 6:

   4.4438e-11  -3.8598e-09   8.2505e-08  -7.5194e-07   3.5931e-06  -9.8904e-06

 Columns 7 through 10:

   1.6243e-05  -1.5708e-05   8.2514e-06   1.0000e+00


## Punto 4d:

Código Matlab usando Semilogy

In [ ]:
n_values = 2:13;

errors_LU = zeros(length(n_values), 1);
errors_CH = zeros(length(n_values), 1);

for k = 1:length(n_values)
    n = n_values(k);

    H = hilb(n);
    b = zeros(n, 1);
    for i = 1:n
        b(i) = 1 / (n + i - 1);
    end
    
    % Solución exacta
    x_exact = zeros(n, 1);
    x_exact(end) = 1;
    
    % Resolver usando LU
    [L, U] = lu(H);
    y_LU = L \ b;
    x_LU = U \ y_LU;

    % Resolver usando Cholesky
    L_chol = chol(H); 
    y_CH = L_chol' \ b;
    x_CH = L_chol \ y_CH;

    % Calcular errores
    errors_LU(k) = norm(x_LU - x_exact);
    errors_CH(k) = norm(x_CH - x_exact);
end
errors_LU(errors_LU == 0) = 1e-16;
% Graficar los errores
figure;
semilogy(n_values, errors_LU, 'r-o', 'LineWidth', 1.5, 'DisplayName', 'LU');
hold on;
semilogy(n_values, errors_CH, 'b-s', 'LineWidth', 1.5, 'DisplayName', 'Cholesky');
grid on;
xlabel('Tamaño de la matriz n');
ylabel('Error');
title('Errores de LU y Cholesky');
legend show;

Para el caso $n\geq 14$ el código falla porque la matriz tiene número de condición muy alto, grafiquemoslo

In [ ]:
n_values = 2:15;
cond_numbers = zeros ( size ( n_values ) );

for i = 1: length ( n_values )
    n = n_values (i);
    H = hilb (n);
    cond_numbers (i) = cond (H);
end

figure ;
semilogy ( n_values , cond_numbers , '-o', 'LineWidth', 1.5 ,'MarkerSize', 8) ;
grid on;
xlabel ('Tamaño de la matriz', 'FontSize', 11) ;
ylabel ('Numero de condicion', 'FontSize', 11) ;

## Punto 5

Código de diferencias finitas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def solve_poisson(n):
    h = 1 / n
    x = np.linspace(0, 1, n + 1)
    
    f = np.sin(2 * np.pi * x[1:-1])  # Evitamos los puntos (T_0 y T_n)
    b = (h ** 2) * f

    main_diag = 2 * np.ones(n - 1)   # np.ones genera un vector de 1's
    off_diag = -1 * np.ones(n - 2)
    
    for i in range(1, n - 1):
        m = off_diag[i - 1] / main_diag[i - 1]
        main_diag[i] = main_diag[i] - m * off_diag[i - 1]
        b[i] =b[i] - m * b[i - 1]

    # Sustitución hacia atrás
    T = np.zeros(n - 1)
    T[-1] = b[-1] / main_diag[-1]
    for i in range(n - 3, -1, -1):
        T[i] = (b[i] - off_diag[i] * T[i + 1]) / main_diag[i]

    T_full = np.zeros(n + 1)
    T_full[1:-1] = T
    return x, T_full

def sol(x):
    return np.sin(2 * np.pi * x) / (4 * np.pi ** 2)

n = 1000
x, T_approx = solve_poisson(n)
T_exact = sol(x)

plt.figure(figsize=(10, 6))
plt.plot(x, T_approx, label="Solución numérica", lw=2)
plt.plot(x, T_exact, label="Solución Exacta", lw=2, linestyle="--")
plt.xlabel("$x$", fontsize=11)
plt.ylabel("$T(x)$", fontsize=11)
plt.legend(fontsize=10)
plt.grid()
plt.show()

error = np.abs(T_exact - T_approx)
print(f"Error máximo: {np.max(error):.6e}")
